<font size=6>**TD 1 : de la chaîne de caractères au sac de mots**</font>

Julien Velcin, Université Lyon 2 - Master Humanités Numériques

Ce TD utilise une partie du tutoriel donné par A. Gramfort et A. Mueller à SciPy 2017
https://github.com/amueller/scipy-2017-sklearn ainsi qu'un article du blog de Thomas Buhrmann https://buhrmann.github.io/tfidf-analysis.html

Il est utilisé dans des travaux pratiques en complément d'un cours magistral qui explique les notions fondamentales de du traitement des données textuelles.

# Differentes manières de représenter des données textuelles

- Chaîne de caractères (string)
- Bag-of-Words (BoW)
- Vector Space Model (VSM)
- Ajouter des méta-données (par ex. catégories grammaticales)
- Représentations plus complexes : arbres syntaxiques, graphes, etc.

Chaque représentation implique une manière différente de *comparer* les documents.

## Un exemple très simple

*"John Doe has bought an apple."*

Point de vue linguistique:

<table style="border:0;">
<tr style="border:0;">
<td style="border:0; white-space:pre; padding:0 100px 0 0px;">"John Doe has bought an apple."</td>
<td style="border:0;"><img src="img/syntaxtree.png" style='height: 150px'/></td>
</tr>
</table>

Point de vue statistique :

<img src="img/bow-illu.png" style='height: 150px'/>

<table style="border:0;">
<tr style="border:0;">
<td style="border:0; white-space:pre; padding:0 100px 0 0px;">"John Doe has bought an apple."</td>
<td style="border:0;">{ apple,<br/><br/> bought,<br/><br/>John_Doe } </td>
</tr>
</table>

## Vers l'espace vectoriel

<img src="img/bow.png" style='height: 400px'/>

On peut déjà observer des problèmes évidents, tel que :

*"Mary asked Fred out."*
<br/>
*"Fred asked Mary out."*

La représentation est la même :
<img src="img/mary.png" style='height: 200px'/>

A partir d'un *ensemble* de documents, on souhaite obtenir l'objet suivant :
<img src="img/termdocmatrix.png" style='height: 300px'/>

Par exemple :
<img src="img/termdocmatrix-2.jpg" style='width: 400px'/>

# Extraire les caractéristiques des textes

En pratique, on utilise le formalisme du "sac de mots" et on suit plusieurs étapes :

<img src="img/bag_of_words.svg" width="100%">


Prenons à présent deux documents, tels que :

In [1]:
X = ["Some say the world will end in fire,",
     "Some say in ice."]

In [2]:
len(X)

2

La librairie contient une fonction qui permet de "vectoriser" un ensemble de textes en prenant en compte un certain nombre de prétraitements (*preprocessing*) couramment employés : mis en minuscule, utilisation de mots-outils, etc.

In [3]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
vectorizer.fit(X)

CountVectorizer()

In [4]:
#help(CountVectorizer)

Observons le vocabulaire automatiquement construit à partir de ces deux textes :

In [5]:
vectorizer.vocabulary_

{'some': 5,
 'say': 4,
 'the': 6,
 'world': 8,
 'will': 7,
 'end': 0,
 'in': 3,
 'fire': 1,
 'ice': 2}

Quelques mots sur la présence de mots tels que "in" ou "the"

Mots les plus fréquents observés dans Harry Potter :

<img src="img/zipf_1.png" width="100%">

Cette forme particulière est appelée "loi de Zipf". Elle a été observée pour la première fois par le linguiste George K. Zipf (1902-1950).

La loi de Zipf indique que si l'on observe un corpus suffisamment grand, la fréquence d'apparition d'un mot est **inversement proportionnelle** à son rang dans la table des fréquences.

Même graphique mais en supprimant les mots-outils anglais :
    
<img src="img/zipf_2.png" width="100%">

D'autres prétraitements sont également disponibles, tels que :

* racinisation (stemming) : trouver la racine des mots, comme dans :
     
> learn: learns, learned, learning...<br/>
> march: marcher, marchera, marcherai...
 
* lemmatisation (lemmatization) : trouver le lemme, comme dans :

> to be: am, are<br/>
> être : suis, sont...  

Le stemming nous aide à :

- rapprocher deux documents sémantiquement liés mais n'utilisant pas exactement les mêmes termes
- réduire la taille du vocabulaire (cf. malédiction de la dimension)

Ok pour :

    adventur: [adventure,adventurer,adventurers,adventures,adventurous]

mais...

    anim: [animal, animals, animation]



En pratique, la **lemmatisation** peut être réalisée via des librairies comme *spacy* ou *stenza* (cf. cours précédent).

Le **stemming** peut être réalisé en amont avec des librairies comme *nltk*, par exemple :

In [6]:
from nltk.stem.snowball import SnowballStemmer

# on commence par créer un nouvel "analyseur" utilisant le stemming (dans la bonne langue)

#stemmer_fr = SnowballStemmer("french")
stemmer_en = SnowballStemmer("english") # à priori, il s'agit ici de l'algorithme de Porter

analyzer_with_stemming = CountVectorizer().build_analyzer()

def stemmed_words_en(doc):
    return (stemmer_en.stem(w) for w in analyzer_with_stemming(doc))

# à présent, on initialisale le "vectorize" avec le nouvel analyseur basé sur le stemming :
stem_vectorizer_en = CountVectorizer(analyzer=stemmed_words_en)

# le reste est identique : construction du vocabulaire (fit) puis de la matrice (transform) :
stem_vectorizer_en.fit(X)
stem_vectorizer_en.transform(X)

<2x9 sparse matrix of type '<class 'numpy.int64'>'
	with 12 stored elements in Compressed Sparse Row format>

Autre prétraitement utile : pouvoir **remplacer** certains motifs trouvés dans les textes

In [7]:
print(X)
str = [x.replace("world", "worlds") for x in X]
print(str)

['Some say the world will end in fire,', 'Some say in ice.']
['Some say the worlds will end in fire,', 'Some say in ice.']


In [8]:
import re
re.sub("[s|S]+\w*", "truc", X[0])

'truc truc the world will end in fire,'

Corriger automatiquement l'orthographe des mots grâce à la distance d'édition (ou distance Levenshtein, proposée en 1965).
Il s'agit de calculer le plus petit nombre d'opérations (insertion, suppression, remplacement) pour passer d'une chaîne à une autre.

In [9]:
# one possible implementation from https://en.wikibooks.org/wiki/Algorithm_Implementation/Strings/Levenshtein_distance#Python
def levenshtein(s1, s2):
    if len(s1) < len(s2):
        return levenshtein(s2, s1)

    # len(s1) >= len(s2)
    if len(s2) == 0:
        return len(s1)

    previous_row = range(len(s2) + 1)
    for i, c1 in enumerate(s1):
        current_row = [i + 1]
        for j, c2 in enumerate(s2):
            insertions = previous_row[j + 1] + 1 # j+1 instead of j since previous_row and current_row are one character longer
            deletions = current_row[j] + 1       # than s2
            substitutions = previous_row[j] + (c1 != c2)
            current_row.append(min(insertions, deletions, substitutions))
        previous_row = current_row
    
    return previous_row[-1]

In [10]:
#levenshtein("chaîne", "chaine")
#levenshtein("chaîne", "chiane")
levenshtein("remerciement", "remerciments")

2

# Matrice documents x termes

## Codage TF

On peut maintenant construire la matrice documents * termes :

In [11]:
X_bag_of_words = vectorizer.transform(X)

In [12]:
X_bag_of_words.shape

(2, 9)

In [13]:
X_bag_of_words

<2x9 sparse matrix of type '<class 'numpy.int64'>'
	with 12 stored elements in Compressed Sparse Row format>

Jetons un oeil au contenu de la matrice :

In [14]:
X_bag_of_words.toarray()

array([[1, 1, 0, 1, 1, 1, 1, 1, 1],
       [0, 0, 1, 1, 1, 1, 0, 0, 0]])

In [15]:
# exemple avec un second corpus

#X = ["Some say the world will end in fire,",
#     "Some say in ice."]
#X_2 = ["Some world say will end in : fire the with new words !,",
#     "Some in ice say."]
X_2 = ["Some world say will end in : fire the with new words end!,",
     "Some in ice say ice ice."]
X_2_bag_of_words = vectorizer.transform(X_2)
X_2_bag_of_words.toarray()

array([[2, 1, 0, 1, 1, 1, 1, 1, 1],
       [0, 0, 3, 1, 1, 1, 0, 0, 0]])

In [16]:
# Le code ci-dessous si on souhaite utiliser un *nouveau* vectorizer (ie nouveau vocabulaire)

#X_2 = ["Some world say will end in : fire the with new words !,",
#     "Some in ice say."]
#vectorizer_2 = CountVectorizer()
#vectorizer_2.fit(X_2)
#X_2_bag_of_words = vectorizer_2.transform(X_2)
#X_2_bag_of_words.toarray()

On observe que la valeur indiquée dans une cellule est le nombre d'occurrences d'un terme dans un document (TF pour *Term Frequency*)

On peut également retrouver le nom des termes du vocabulaire :

In [17]:
vectorizer.get_feature_names_out()

array(['end', 'fire', 'ice', 'in', 'say', 'some', 'the', 'will', 'world'],
      dtype=object)

On peut retrouver les attributs (features) utilisés dans les documents :

In [18]:
vectorizer.inverse_transform(X_bag_of_words)

[array(['end', 'fire', 'in', 'say', 'some', 'the', 'will', 'world'],
       dtype='<U5'),
 array(['ice', 'in', 'say', 'some'], dtype='<U5')]

## Représentation TFxIDF
A la place du nombre d'occurrences (TF), on peut utiliser une autre mesure qui prend en compte la rareté d'un mot dans le corpus :

$$ tf_{t,d} \times idf_{t} $$

avec $tf_{t,d}$ le nombre d'occurrences de $t$ dans $d$

et $idf_{t} = \log \frac{N}{df_t}$ ($N$ est le nombre total de documents)

In [19]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer()
tfidf_vectorizer.fit(X)

TfidfVectorizer()

In [20]:
import numpy as np
np.set_printoptions(precision=2)

X_TFxIDF = tfidf_vectorizer.transform(X)
print(X_TFxIDF.toarray())

[[0.39 0.39 0.   0.28 0.28 0.28 0.39 0.39 0.39]
 [0.   0.   0.63 0.45 0.45 0.45 0.   0.   0.  ]]


# Comparaison de deux textes

Les distances usuelles (ex. euclidenne) ne sont pas adaptées.

Dans les espaces à **beaucoup de dimensions** :

Pourquoi les banquiers n'ont jamais de lingots sphériques ?

Pourquoi les marchands d'oranges occupent beaucoup de place pour empiler peu d'oranges ?

http://www.brouty.fr/Maths/sphere.html (see "Curiosités du calcul")

Malédiction de la dimension (curse of dimensionality)

Richard E. Bellman (1920-1984): les hypervolumes sont presque vides!

<img src="img/curse.png" style='height: 400px'/>

Un volume avec $dim=d$ a besoin de $10^d$ données pour peupler équitablement l'espace.

## Produit scalaire et cosinus

$\vec{x}$ et $\vec{y}$ sont deux vecteurs dans le VSM.

Cosine est une mesure de **similarité** calculée sur l'angle formé par les deux vecteurs :

$$cos(\vec{x},\vec{y}) = \frac{\vec{x}.\vec{y}}{||\vec{x}||_2 \times ||\vec{y}||_2}$$

Elle prend une valeur entre 0 (rien en commun) et 1 (même vecteurs à une constante près).

## Interprétation géométrique

<img src="img/cosine.png" style='height: 300px'/>

## Exemple : distribution des mots les plus fréquents dans Harry Potter

Commençons par lire les données :

In [21]:
import os

with open(os.path.join("data", "Harry_Potter_1.txt")) as f:
    lines = [line.strip() for line in f.readlines()]

# version de base :
tf_vectorizer = CountVectorizer()

# version avec une liste de mots outils fournie
sw = ["they", "were", "to", "and"] # etc. (il est préférable de charger une liste depuis un fichier)
tf_vectorizer = CountVectorizer(stop_words=sw)
# si vous voulez utiliser une liste pré-construite :
#tf_vectorizer = CountVectorizer(stop_words="english")
# si vous voulez utiliser l'encodage TFxIDF :
#tfidf_vectorizer = TfidfVectorizer()

tf_vectorizer.fit(lines)

# montrer l'intégralité du vocabulaire (peut être long) :
#tf_vectorizer.vocabulary_

CountVectorizer(stop_words=['they', 'were', 'to', 'and'])

In [33]:
# si on veut tester en même temps avec le stemming et la suppression de mots-outils tirés d'un fichier :

## on charge les mots-outils depuis un fichier :
#with open(os.path.join("stop", "Stop-words-en.txt")) as f:
#    my_sw_en = [line.strip() for line in f.readlines()]

## analyseur qui réalise le stemming et la suppression des mots-outils
#def stem_sw_words_en(doc):
#    return (stemmer_en.stem(w) for w in analyzer_with_stemming(doc) if w not in my_sw_en)

#stem_sw_vectorizer_en = CountVectorizer(analyzer=stem_sw_words_en)
#stem_sw_vectorizer_en.fit(lines)

#X_hp_stem_sw = stem_sw_vectorizer_en.transform(lines)
#features_hp_stem_sw = stem_sw_vectorizer_en.get_feature_names_out()
#tf_sum_stem_sw = X_hp_stem_sw.sum(axis=0)
#tf_sum_stem_sw = tf_sum_stem_sw.tolist()[0] # conversion en liste

#print_feats(tf_sum_stem_sw, features_hp_stem_sw)


,word,value
0,harri,1326
1,ron,429
2,look,413
3,hagrid,370
4,hermion,270
5,go,269
6,one,269
7,back,267
8,know,245
9,get,236


In [23]:
# puis on construit la matrice Documents x Termes basée sur le vocabulaire

X_hp = tf_vectorizer.transform(lines)
features_hp = tf_vectorizer.get_feature_names_out()
#len(features_hp)
#features_hp[0:10]

On peut utiliser quelques fonctions pour afficher le vecteur pour un document en particulier :

In [24]:
from scipy.sparse import find, csr_matrix
import pandas as pd

# des options permettent de limiter (ou non) le nombre de lignes/colonnes affichées
# par exemple :
# pd.set_option('display.max_rows', None)

# cette fonction permet d'afficher une "jolie" représentation du vecteur v
# ARGS :
#   v : le vecteur à afficher (par ex. une ligne de la matrice X)
#   features : le vocabulaire
#   top_n : le nombre de mots maximum à afficher
def print_feats(v, features, top_n = 30):
    _, ids, values = find(v)
    feats = [(ids[i], values[i], features[ids[i]]) for i in range(len(list(ids)))]
    top_feats = sorted(feats, key=lambda x: x[1], reverse=True)[0:top_n]
    return pd.DataFrame({"word" : [t[2] for t in top_feats], "value": [t[1] for t in top_feats]})   

In [25]:
print(lines[4])

print_feats(X_hp[4], features_hp, top_n=35)

Mr. and Mrs. Dursley, of number four, Privet Drive, were proud to say that they were perfectly normal, thank you very much. They were the last people you’d expect to be involved in anything strange or mysterious, because they just didn’t hold with such nonsense.


,word,value
0,you,2
1,anything,1
2,be,1
3,because,1
4,didn,1
5,drive,1
6,dursley,1
7,expect,1
8,four,1
9,hold,1


Afficher les mots les plus fréquents :

In [26]:
n_docs, n_terms = X_hp.shape

# on fait la somme sur toutes les lignes pour chacun des mots
tf_sum = X_hp.sum(axis=0)
tf_sum = tf_sum.tolist()[0] # conversion en liste

print_feats(tf_sum, features_hp)

,word,value
0,the,3627
1,he,1759
2,harry,1326
3,of,1267
4,it,1186
5,was,1186
6,you,1037
7,in,967
8,his,937
9,said,794


Comparer des documents revient à comparer les valeurs de deux colonnes (vecteurs).

In [27]:
import math
from scipy.linalg import norm

# fonction calculant le cosinus entre deux vecteurs
def cosinus(i, j):
        # numérateur : <i.j>
    num = i.dot(j.transpose())[0,0]
        # dénominateur : ||i||_2 * ||j||_2
    den = norm(i.todense()) * norm(j.todense())
    if (den>0): # on vérifie que le dénominateur n'est pas nul
        return (num/den)
    else:
        return 0

In [28]:
print(print_feats(X_hp[5], features_hp, top_n=40))
print(print_feats(X_hp[10], features_hp, top_n=40))

         word  value
0         the      4
1         was      4
2          of      3
3      called      2
4     dursley      2
5         had      2
6          he      2
7          in      2
8        neck      2
9        very      2
10      which      2
11   although      1
12     amount      1
13        any      1
14   anywhere      1
15         as      1
16      beefy      1
17        big      1
18     blonde      1
19        boy      1
20       came      1
21    craning      1
22        did      1
23   director      1
24     drills      1
25     dudley      1
26   dursleys      1
27     fences      1
28      finer      1
29       firm      1
30     garden      1
31  grunnings      1
32     hardly      1
33       have      1
34        her      1
35      large      1
36       made      1
37        man      1
38         mr      1
39        mrs      1
        word  value
0         he      2
1         as      1
2     backed      1
3        car      1
4   chortled      1
5      drive      1

In [29]:
cosinus(X_hp[5], X_hp[10])
#cosinus(X_hp[5], X_hp[5])
#cosinus(X_hp[5], 3*X_hp[5])
#cosinus(X_hp[2], X_hp[5])
#cosinus(X_hp[2], X_hp[0])

0.2808772073524269

# Créer son propre moteur de recherche

In [ ]:
#query = ['privet', 'drive', 'dursley']
#query = ['privet', 'privet', 'drive', 'dursley']
query = ['chess', 'chess', 'chess', 'harry', 'play']
#query = ['1473', 'aaaaarrrgh']

indexes = [np.where(features_hp == q)[0][0] for q in query if q in features_hp]
print(indexes)

On construit un vecteur de la même taille que le vocabulaire. Il est initialisé à zéro, puis on y met la valeur 1 pour les termes de la requête.

In [ ]:
query_vec = np.zeros(n_terms)

# pour mettre 1 aux index des mots-clefs
#query_vec[indexes] = 1

# alternative pour pouvoir mettre plus que 1 en répétant les mots-clefs
for tt in indexes:
    query_vec[tt] += 1

In [ ]:
#len(query_vec)
query_vec[3658:3670]
#query_vec[810:830]
#query_vec[0:10]
#query_vec[1410:1420]

On peut vérifier que le vecteur requête contient bien 3 éléments non nuls.

In [ ]:
query_vec = csr_matrix(query_vec)
query_vec.sum()

Calcul du cosinus vis-à-vis de la requête pour 1 document :

In [ ]:
cosinus(X_hp[4], query_vec)

On automatise en calculant pour tous les docs et on triant le résultat :

In [ ]:
# fonction qui crée un dictionnaire associant le cosinus à chaque document
# puis le trie de manière décroissante

def search(q, X):
    cc = {i: cosinus(X[i], q) for i in range(n_docs)}
    cc = sorted(cc.items(), key=lambda x: x[1], reverse=True)
    return cc

In [ ]:
result = search(query_vec, X_hp)

In [ ]:
result[0:10]

On ne retient que les dix premiers résultats (par exemple).

In [ ]:
nb_top_docs = 10
top_docs = [r for (r,v) in result[0:nb_top_docs]]
print(top_docs)

Pour finir, on peut afficher les textes les plus pertinents pour la requête :

In [ ]:
for i, td in zip(range(nb_top_docs), top_docs):
    #print(top_feats_in_doc(X_hp, features_hp, td))
    print("%s (%s): %s" % (i+1, td, lines[td]))